In [102]:
import twitter
import json
import os
import time
#search function is under ./Code/search.py
import sys
sys.path.append("./Code/") #add the path with the code so that it can be imported
import search


## LoadFile() opens the specified file and returns a json object of its contents


In [104]:
def LoadFile(pathName):
    if os.path.isfile(pathName):
        file = open(os.path.join(pathName), 'r')
        return json.load(file)
    else: print("File not in directory")

In [105]:
# Will sleep for n seconds

def trusty_sleep(n):
    start = time.time()
    while (time.time() - start < n):
        time.sleep(n - (time.time() - start))

## Extract

In [106]:
#Takes in a list of statuses, extracts info from the status and places into dictionart
def extract_data(statuses, mydictionary, day_number):
    '''
    Parameters:
    Statuses(list) - Iterates over a list of statuses
    mydictionary - Dictionary to be filled in
    day_number - Day number to be filled in dictionary
    
    I excluded the code for searching all the statuses for a given day, since Logan is doing this.
    All this function serves to do is extract the information and fill in the corresponding fields
    in the dictionary.
    
    '''
    day_number = 'day' + str(day_number)
    for i in statuses:
        try:
            mydictionary[day_number]['created_at'].append(i['created_at'])
            mydictionary[day_number]['followers_count'].append(i['user']['followers_count'])
            mydictionary[day_number]['retweet_count'].append(i['retweet_count'])
            # Increments the count of lang if the lang exists, else intializes it with value 1
            mydictionary[day_number]['lang'][i['user']['lang']] = mydictionary[day_number]['lang'].get(i['user']['lang'], 0) + 1
            # favorite_count is nullable, so it may not exist
            if i.get('favorite_count', False):
                mydictionary[day_number]['favorite_count'].append(i['favorite_count'])
        except (KeyError):
            continue
            
    #print(json.dumps(tempdict, indent = 1))

## Search

In [107]:
def DaySearch(htDictionary, tagIndex, startDay, endDay):
    '''
    htDictionary: The hashtags dictionary from file
    tagIndex: The "index" of the current hashtag being searched
    startDay: The starting calendar day for the search
    endDay: The ending calendar day for the search
    
    I'm calling searchOverRange function per day instead 
    of using it for multiple days to implement Sleep.'''
    
    holdResults = []
    currentHashtag = list(htDictionary.items())[tagIndex]
    for day, details in currentHashtag[1].items():
        try:
            results = search.searchOverRange(currentHashtag[0], 2018, 10, startDay, 2018, 10, startDay+1, debug=1)
        except:
            print("\nStopped at day {}".format(startDay))
            break
        if not results: break
        try:
            #print("{} requests remaining\n".format(results[-1]['metadata']['x-rate-limit-remaining']))
            numRequests = results[-1]['metadata']['x-rate-limit-remaining']
            print("Requests remaining: {}".format(numRequests))
        except: 
            print("\nStopped at day {}".format(startDay))
            break
            
        holdResults.append(results[0])
        startDay += 1
        if startDay >= endDay: break
        elif int(results[-1]['metadata']['x-rate-limit-remaining']) == 0: break    
    return holdResults

## Search/Extract 
`startingIndex` and `endingIndex` are used to split up the searches.

for example: `startingIndex = 100` and `endingIndex = 199` will search/extract the 100-199 hashtags in the hashtags dictionary


In [ ]:
# this will hold the hashtags we extract in the same format as the hashtags dictionary

##############################################################################
##############################################################################
#
# RE-RUNNING THIS CELL WILL DELETE ALL PREVIOUS DATA SAVED IN DATADICTIONARY #
#
##############################################################################
##############################################################################

dataDictionary = {}

In [108]:
###########################
# CHANGE PARTITION NUMBER #
###########################
hashtags = LoadFile(os.path.join('Data', 'Tweets-Hashtags-Partition1.json'))


# To search in parts
startingIndex = 0
endingIndex = 20 # the max would be len(hashtags)


for tagIndex in range(startingIndex, endingIndex):
    tagString = list(hashtags.items())[tagIndex][0]
    dayDictionary = dict(list(hashtags.items())[tagIndex][1])
    print("\nSearching hashtag: {}\nindex {}\n".format(tagString, tagIndex))
    
    daycount = 0
    for x in range(12, 19):
        #print(tagString)
        #print(dayDictionary.keys(), end = '\n\n')
        
        resultList = DaySearch(hashtags, tagIndex, x, x+1)
        if (resultList != [[]]) and (resultList != []):
            dayDictionary['day' + str(daycount)]['occurrences'] = resultList[0]['numberOfTweets']
            extract_data(resultList[0]['tweets'], dayDictionary, daycount)
            dataDictionary[tagString] = dict(dayDictionary)
            
        if (resultList == []) or (int(resultList[-1]['metadata']['x-rate-limit-remaining']) == 0):
            print("Sleeping...")
            trusty_sleep(930)
            print("Resuming :)\n")
        
        
        daycount +=1



Searching hashtag: رنيه
index 0

Error 429: ('Too Many Requests: Request cannot be served ', "due to the application's rate limit having ", 'been exhausted for the resource')

Stopped at day 12
Sleeping...


KeyboardInterrupt: 

## Testing

In [109]:
print(len(dataDictionary))#['トレクル']['day0']['retweet_count']))

25


In [110]:
for htag, dayDict in dataDictionary.items():
    print(htag)
    for day, details in dayDict.items():
        print("  {} occurrences: {}".format(day, details['occurrences']))

رنيه
  day0 occurrences: 43
  day1 occurrences: 45
  day2 occurrences: 42
  day3 occurrences: 23
  day4 occurrences: 97
  day5 occurrences: 35
  day6 occurrences: 13
アズールレーン
  day0 occurrences: 44
  day1 occurrences: 29
  day2 occurrences: 37
  day3 occurrences: 28
  day4 occurrences: 28
  day5 occurrences: 32
  day6 occurrences: 25
トレクル
  day0 occurrences: 16600
  day1 occurrences: 0
  day2 occurrences: 0
  day3 occurrences: 0
  day4 occurrences: 0
  day5 occurrences: 0
  day6 occurrences: 0
pornlife
  day0 occurrences: 7
  day1 occurrences: 0
  day2 occurrences: 3
  day3 occurrences: 1
  day4 occurrences: 0
  day5 occurrences: 1
  day6 occurrences: 2
NuVolución
  day0 occurrences: 260
  day1 occurrences: 42
  day2 occurrences: 1
  day3 occurrences: 1
  day4 occurrences: 0
  day5 occurrences: 0
  day6 occurrences: 0
ゆららぎBot
  day0 occurrences: 22
  day1 occurrences: 21
  day2 occurrences: 28
  day3 occurrences: 18
  day4 occurrences: 25
  day5 occurrences: 27
  day6 occurrences: 19
De

In [111]:
# The first bracket [2] is index, 
# second bracket [1] is hashtag's days dictionary, or [0] for the hashtag string

#print(json.dumps(dataDictionary, indent = 1))
print(*list(dataDictionary.items())[10][1]['day3']['lang'].items(), sep = '\n', end = '\n\n')

('en', 628)
('es', 12)
('ko', 6)
('pt', 51)
('ja', 60)
('ar', 25)
('it', 4)
('fr', 6)
('tr', 2)
('zh-CN', 1)
('en-gb', 2)
('sv', 2)



## Stores dataDictionary

In [112]:
def StoreHashtags(pathName, htDictionary): 
    newPath = pathName[:-5] + "-data.json"

    with open(newPath, 'w') as f:    
        json.dump(htDictionary, f)
        f.close()
    print("Stored data in {}".format(newPath))

## Change partition number in case you want to gather data at different intervals or if you reset dataDictionary to ensure you dont overwrite previously saved data

In [114]:
###########################
# CHANGE PARTITION NUMBER #
###########################

StoreHashtags(os.path.join('Data', 'Partition1.2.json'), dataDictionary)

Stored data in Data/Partition1.2-data.json
